In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import scanpy as sc
import SpaGCN as spg
import torch
from sklearn import metrics
import cv2
import matplotlib.pyplot as plt
import random    

In [4]:
def process_sample(sample_name, base_path, output_path):
    base_path = Path(base_path)
    output_path = Path(output_path)
    dir_input = base_path / sample_name
    dir_output = output_path / sample_name
    dir_output.mkdir(parents=True, exist_ok=True)
    n_clusters = 5 if sample_name in ['151669', '151670', '151671', '151672'] else 7
    adata = sc.read_10x_h5(dir_input / 'filtered_feature_bc_matrix.h5')
    adata.var_names_make_unique()

    spatial = pd.read_csv(dir_input / 'spatial' / 'tissue_positions_list.csv', sep=",", header=None, na_filter=False, index_col=0)

    adata.obs['x1'] = spatial[1]
    adata.obs['x2'] = spatial[2]
    adata.obs['x3'] = spatial[3]
    adata.obs['x4'] = spatial[4]
    adata.obs['x5'] = spatial[5]

    adata = adata[adata.obs['x1'] == 1]
    adata.var_names = [i.upper() for i in list(adata.var_names)]
    adata.var['genename'] = adata.var.index.astype('str')

    img = cv2.imread("/Users/abdullahalsakib/Downloads/fydp/finalone/autoencoder/histology.tif")

    adata.obs['x_array'] = adata.obs['x2']
    adata.obs['y_array'] = adata.obs['x3']
    adata.obs['x_pixel'] = adata.obs['x4']
    adata.obs['y_pixel'] = adata.obs['x5']
    x_pixel = adata.obs['x_pixel'].tolist()
    y_pixel = adata.obs['y_pixel'].tolist()
    img_new = img.copy()
    for i in range(len(x_pixel)):
        x, y = x_pixel[i], y_pixel[i]
        img_new[int(x - 20):int(x + 20), int(y - 20):int(y + 20), :] = 0

    cv2.imwrite(f'{dir_output}/sample_map.jpg', img_new)
    b = 49
    a = 1
    adj = spg.calculate_adj_matrix(x=x_pixel, y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=a, histology=True)
    np.savetxt(f'{dir_output}/adj.csv', adj, delimiter=',')
    spg.prefilter_genes(adata, min_cells=3)
    spg.prefilter_specialgenes(adata)
    sc.pp.normalize_per_cell(adata)
    sc.pp.log1p(adata)
    p = 0.5
    spg.test_l(adj, [1, 10, 100, 500, 1000])
    l = spg.find_l(p=p, adj=adj, start=100, end=500, sep=1, tol=0.01)
    n_clusters = n_clusters
    r_seed = t_seed = n_seed = 100
    res = spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed,
                         t_seed=t_seed, n_seed=n_seed)
    clf = spg.SpaGCN()
    clf.set_l(l)
    random.seed(r_seed)
    torch.manual_seed(t_seed)
    np.random.seed(n_seed)
    clf.train(adata, adj, init_spa=True, init="louvain", res=res, tol=5e-3, lr=0.05, max_epochs=200)
    y_pred, prob = clf.predict()
    categories = list(map(str, np.unique(y_pred)))
    adata.obs['pred'] = pd.Categorical(y_pred.astype(str), categories=categories)
    adj_2d = spg.calculate_adj_matrix(x=adata.obs['x_array'], y=adata.obs['y_array'], histology=False)
    refined_pred = spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs['pred'].tolist(), dis=adj_2d, shape="hexagon")
    refined_pred_series = pd.Series(refined_pred)
    adata.obs['refined_pred'] = pd.Categorical(refined_pred_series.astype(str), categories=categories)
    plot_color = ["#F56867", "#FEB915", "#C798EE", "#59BE86", "#7495D3", "#D1D1D1", "#6D1A9C", "#15821E", "#3A84E6",
                  "#997273", "#787878", "#DB4C6C", "#9E7A7A", "#554236", "#AF5F3C", "#93796C", "#F9BD3F", "#DAB370",
                  "#877F6C", "#268785"]
    domains = "pred"
    num_celltype = len(adata.obs[domains].unique())
    adata.uns[domains + "_colors"] = list(plot_color[:num_celltype])
    ax = sc.pl.scatter(adata, alpha=1, x="y_pixel", y="x_pixel", color=domains, title=domains, color_map=plot_color,
                       show=False, size=100000 / adata.shape[0])
    ax.set_aspect('equal', 'box')
    ax.axes.invert_yaxis()
    plt.savefig(f"{dir_output}/pred.png", dpi=300)
    plt.close()
    domains = "refined_pred"
    num_celltype = len(adata.obs[domains].unique())
    adata.uns[domains + "_colors"] = list(plot_color[:num_celltype])
    ax = sc.pl.scatter(adata, alpha=1, x="y_pixel", y="x_pixel", color=domains, title=domains, color_map=plot_color,
                       show=False, size=100000 / adata.shape[0])
    ax.set_aspect('equal', 'box')
    ax.axes.invert_yaxis()
    plt.savefig(f"{dir_output}/refined_pred.png", dpi=300)
    plt.close()

    return adata

sample_list = ['151673']  
ARI_list = []

for sample_name in sample_list:
    adata = process_sample(sample_name, '/Users/abdullahalsakib/Downloads/fydp/finalone/others/SpaGcn',
                         '/Users/abdullahalsakib/Downloads/fydp/finalone/others/SpaGcn/result')

adata


/Users/abdullahalsakib/anaconda3/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/abdullahalsakib/anaconda3/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Calculateing adj matrix using histology image...
Var of c0,c1,c2 =  33.30687202862215 174.55510595352243 46.84205750749746
Var of x,y,z =  5606737.526317932 4468793.817921193 5606737.526317932
l is  1 Percentage of total expression contributed by neighborhoods: 0.0
l is  10 Percentage of total expression contributed by neighborhoods: 0.0
l is  100 Percentage of total expression contributed by neighborhoods: 0.23831093311309814
l is  500 Percentage of total expression contributed by neighborhoods: 28.014718548210983
l is  1000 Percentage of total expression contributed by neighborhoods: 153.882049263866
L= 100 P= 0.23831
L= 101 P= 0.24709
L= 102 P= 0.25606
L= 103 P= 0.2652
L= 104 P= 0.27454
L= 105 P= 0.28405
L= 106 P= 0.29376
L= 107 P= 0.30365
L= 108 P= 0.31374
L= 109 P= 0.32402
L= 110 P= 0.33449
L= 111 P= 0.34515
L= 112 P= 0.35601
L= 113 P= 0.36707
L= 114 P= 0.37832
L= 115 P= 0.38978
L= 116 P= 0.40144
L= 117 P= 0.41329
L= 118 P= 0.42536
L= 119 P= 0.43763
L= 120 P= 0.4501
L= 121 P= 0.46

AnnData object with n_obs × n_vars = 3639 × 19130
    obs: 'x1', 'x2', 'x3', 'x4', 'x5', 'x_array', 'y_array', 'x_pixel', 'y_pixel', 'n_counts', 'pred', 'refined_pred'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'
    uns: 'log1p', 'pred_colors', 'refined_pred_colors'

In [5]:
from sklearn import metrics
df_meta = pd.read_csv('/Users/abdullahalsakib/Downloads/fydp/finalone/others/SpaGcn/151673/metadata.tsv', sep='\t')
df_meta_layer = df_meta['layer_guess']
adata.obs['Ground_Truth'] = df_meta_layer.values
adata = adata[~pd.isnull(adata.obs['Ground_Truth'])]    

In [6]:
ARI = metrics.adjusted_rand_score(adata.obs['refined_pred'], adata.obs['Ground_Truth'])
adata.uns['ARI'] = ARI
print('ARI:', ARI)

ARI: 0.455702591256653


/var/folders/zt/c6n2w1f53ngbmqr21hfs1yhw0000gn/T/ipykernel_86343/1526278021.py:2: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['ARI'] = ARI


In [8]:
adata

AnnData object with n_obs × n_vars = 3611 × 19130
    obs: 'x1', 'x2', 'x3', 'x4', 'x5', 'x_array', 'y_array', 'x_pixel', 'y_pixel', 'n_counts', 'pred', 'refined_pred', 'Ground_Truth'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'
    uns: 'log1p', 'pred_colors', 'refined_pred_colors', 'ARI'